In [1]:
# Change directory to the root of the project
import os 
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes


In [2]:
from applications.dots_and_boxes.NNmodels.linear_attention_transformer import LinearAttentionTransformerInitParams
import torch

# Initialize parameters

## Model parameters
model_type = 'linear_attention_transformer'
model_params: LinearAttentionTransformerInitParams = {
    'num_rows': 2,
    'num_cols': 2,
    'embed_dim': 32,
    'ff_dim': 128,
    'num_heads': 4,
    'num_layers': 1
}
device = torch.device('mps')
model_name = 'dots_and_boxes_linear_attention_transformer'

## Initialize new model
load_model = None
load_model_params = {}

## Optimizer parameters
optimizer_type = 'adam'
optimizer_params = {
    'lr': 1e-2,
    'betas': (0.9, 0.999),
    'eps': 1e-8,
    'weight_decay': 1e-4,
    'amsgrad': False
}

## Learning scheduler parameters
lr_scheduler_type = 'plateau'
lr_scheduler_params = {
    'factor': 0.5,
    'patience': 25,
    'cooldown': 50,
    'min_lr': 1e-6
}

## Training parameters
training_method = 'supervised'
trainer_params = {}
training_params = {
    'epochs': 1000,
    'batch_size': 256,
    'eval_freq': 25,
    'checkpoint_freq': 50,
    'mask_illegal_moves': False,
    'mask_value': -20.0, # Doesn't matter when mask_illegal_moves is False
    'checkpoint_dir': 'checkpoints',
    'start_at': 1
}

## Load replay buffer from wandb
load_replay_buffer = 'from_file'
load_replay_buffer_params = {
    'path': f'applications/dots_and_boxes/training_data/dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_DABSimpleTensorMapping_minimax.pkl',
    'device': device
}
# load_replay_buffer = 'from_wandb'
# load_replay_buffer_params = {
#     'project': 'AlphaZero-DotsAndBoxes',
#     'artifact_name': f'dots_and_boxes_{model_params["num_rows"]}x{model_params["num_cols"]}_SimpleTensorMapping_minimax',
#     'artifact_version': 'latest'
# }


In [ ]:
# Initialize wandb run
import wandb

run_name = 'Linear Attention Transformer'
notes = 'Linear Attention Transformer for 2 x 2 board.'

config = {
    'model_type': model_type,
    'model_params': model_params,
    'optimizer_type': optimizer_type,
    'optimizer_params': optimizer_params,
    'lr_scheduler_type': lr_scheduler_type,
    'lr_scheduler_params': lr_scheduler_params,
    'training_method': training_method,
    'trainer_params': trainer_params,
    'training_params': training_params
}

run = wandb.init(
    project='AlphaZero-DotsAndBoxes',
    name=run_name,
    config=config,
    notes=notes,
    group=f'{training_method} training on {model_params["num_rows"]}x{model_params["num_cols"]} board'
)
# run = None

In [4]:
# Perform training

from applications.dots_and_boxes.train import train

model_interface = train(
    num_rows = model_params["num_rows"],
    num_cols = model_params["num_cols"],
    model_type=model_type,
    model_params=model_params,
    device=device,
    model_name=model_name,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    lr_scheduler_type=lr_scheduler_type,
    lr_scheduler_params=lr_scheduler_params,
    training_method=training_method,
    trainer_params=trainer_params,
    training_params=training_params,
    load_model=load_model,
    load_model_params=load_model_params,
    load_replay_buffer=load_replay_buffer,
    load_replay_buffer_params=load_replay_buffer_params,
    wandb_run=run
)

/Users/eohjelle/Documents/2025-dots-and-boxes/dots-and-boxes/core/data_structures.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_loca

Epoch 1/200, Train Loss: 2.5687, Val Loss: 2.5024, Learning rate: 0.010000
Epoch 2/200, Train Loss: 2.3085, Val Loss: 2.2453, Learning rate: 0.010000
Epoch 3/200, Train Loss: 2.2231, Val Loss: 2.1863, Learning rate: 0.010000
Epoch 4/200, Train Loss: 2.1588, Val Loss: 2.1505, Learning rate: 0.010000
Epoch 5/200, Train Loss: 2.1528, Val Loss: 2.1272, Learning rate: 0.010000
Epoch 6/200, Train Loss: 2.1367, Val Loss: 2.1305, Learning rate: 0.010000
Epoch 7/200, Train Loss: 2.1145, Val Loss: 2.1243, Learning rate: 0.010000
Epoch 8/200, Train Loss: 2.1296, Val Loss: 2.1317, Learning rate: 0.010000
Epoch 9/200, Train Loss: 2.1020, Val Loss: 2.1236, Learning rate: 0.010000
Epoch 10/200, Train Loss: 2.0630, Val Loss: 2.0307, Learning rate: 0.010000
Epoch 11/200, Train Loss: 2.0052, Val Loss: 1.9721, Learning rate: 0.010000
Epoch 12/200, Train Loss: 1.9741, Val Loss: 1.9955, Learning rate: 0.010000
Epoch 13/200, Train Loss: 1.9578, Val Loss: 1.9555, Learning rate: 0.010000
Epoch 14/200, Train L

KeyboardInterrupt: 

In [5]:
run.finish()

Minimax_draw_rate,▆█▁▁▁▁█▁
Minimax_loss_rate,█▁▂▁▁▁▁▁
Minimax_score,▁▂████▂█
Minimax_win_rate,▁▁████▁█
RandomAgent_draw_rate,▇▄▁▃█▇▆▄
RandomAgent_loss_rate,█▆█▆▆▃▁▃
RandomAgent_score,▁▆█▇▁▅▇█
RandomAgent_win_rate,▂▆█▆▁▃▅▆
epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
learning_rate,███████████████████████████████████████▁
train_loss,▄█▇▆▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [6]:
def print_model_parameters(model):
    """
    Print the total number of parameters in a PyTorch model,
    with a breakdown of trainable vs non-trainable parameters.
    
    Args:
        model: PyTorch model
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%})")
    print(f"Non-trainable parameters: {non_trainable_params:,} ({non_trainable_params/total_params:.2%})")
    
    # Optional: Print parameters by layer
    print("\nParameters by layer:")
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,} parameters")

# Example usage
print_model_parameters(model_interface.model)

Total parameters: 85,349
Trainable parameters: 85,349 (100.00%)
Non-trainable parameters: 0 (0.00%)

Parameters by layer:
layers.0.norm.weight: 12 parameters
layers.0.norm.bias: 12 parameters
layers.0.linear.weight: 768 parameters
layers.0.linear.bias: 64 parameters
layers.1.norm.weight: 64 parameters
layers.1.norm.bias: 64 parameters
layers.1.linear.weight: 8,192 parameters
layers.1.linear.bias: 128 parameters
layers.2.norm.weight: 128 parameters
layers.2.norm.bias: 128 parameters
layers.2.linear.weight: 32,768 parameters
layers.2.linear.bias: 256 parameters
layers.3.norm.weight: 256 parameters
layers.3.norm.bias: 256 parameters
layers.3.linear.weight: 32,768 parameters
layers.3.linear.bias: 128 parameters
layers.4.norm.weight: 128 parameters
layers.4.norm.bias: 128 parameters
layers.4.linear.weight: 8,192 parameters
layers.4.linear.bias: 64 parameters
policy_head.weight: 768 parameters
policy_head.bias: 12 parameters
value_head.weight: 64 parameters
value_head.bias: 1 parameters
